In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2 as cv
from skimage import exposure
from tqdm import tqdm

In [2]:
def normalize(X):
    for i in tqdm(range(X.shape[-1])):
        X[..., i] = exposure.rescale_intensity(X[..., i], out_range=(-1, 1))
        X[..., i] = exposure.equalize_adapthist(X[..., i])
    return X

In [3]:
files = glob.glob("keren/*.npz")
files = np.sort(np.array(files))

In [4]:
def extract_cells(image, mask, ctypes, cells, labels):
    cells_ind = np.unique(mask[:, :, 1])
    cell_mask = mask[:, :, 1]
    cell_size=64
    for cell_ind in tqdm(cells_ind):
        label = ctypes[cell_ind]
        labels.append(label)
        
        x, y = np.where(cell_mask==cell_ind)
        if len(x)==0 or len(y)==0:
            continue
        x1, y1, x2, y2 = x.min(), y.min(), x.max(), y.max()
        padding = cell_size // 2
        x1 = max(0, x1-padding)
        y1 = max(0, y1-padding)
        x2 = min(mask.shape[0], x2+padding)
        y2 = min(mask.shape[1], y2+padding)
        
        cell = image[x1:x2, y1:y2]
        cell_image = cv.resize(cell, (cell_size, cell_size))
        cells.append(cell_image)
            
    return cells, labels

In [5]:

data = np.load(files[0], allow_pickle=True)
image = data["X"][0]
mask = data["y"][0]
eq_X = normalize(image)
ctypes = data["cell_types"].item()
cells, labels = extract_cells(image, mask, ctypes, cells, labels)

cells = np.array(cells)
labels = np.array(labels)

100%|███████████████████████████████████████████| 51/51 [00:46<00:00,  1.10it/s]


NameError: name 'cells' is not defined

In [ ]:
cells, labels = process_data(files)

In [ ]:
len(labels)

In [ ]:
np.save("cell_channels.npy", cells, allow_pickle=True)
np.save("cell_labels.npy", labels, allow_pickle=True)